# 3. 데이터 불러오기

딥러닝을 포함한 머신러닝의 근원은 데이터다. 따라서 데이터의 수집,가공,사용 방법에 따라 모델 성능이 크게 달라질 수 있으며 데이터의 형태는 매우 다양하기 떄문에 데이터를 잘 불러오는 것은 가장 중요한 단계 중 하나다. 

In [ ]:
import torch 
import torchvision 
import torchvision.transforms as tr  #이미지 전처리 기능
from torch.utils.data import DataLoader, Dataset # 데이터를 모델에 사용할 수 있도록 정리해 주는 라이브러리 
import numpy as np 

# 3.1 파이토치 제공 데이터 사용

In [ ]:
# https://pytorch.org/docs/stable/torchvision/transforms.html 다양한 전처리 방법 공부 

transf = tr.Compose([tr.Resize(16), tr.ToTensor()]) # 16x16으로 이미지 크기 변환 후 텐서 타입으로 변환한다. 
   

In [ ]:
# torchvision.datasets에서 제공하는 CIFAR10 데이터셋 불러오기
# root에는 다운로드 받을 경로를 입력하기
# train = True 이면 학습 데이터를 불러오고 False이면 테스트 데이터를 불러온다
# 미리 선언한 전처리를 사용하기 위해 transform=transf 작성

trainset = torchvision.datasets.CIFAR10(root = './data', train=True, download=True, transform=transf)
testset = torchvision.datasets.CIFAR10(root = './data', train=False, download=True, transform=transf)


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# 일반적으로 데이터셋은 이미지와 라벨이 동시에 들어있는 튜플형태이다 (이미지, 라벨)
# traninset[0]은 학습 데이터의 이미지 한장과 라벨 하나가 저장되어 있다.
# 즉 trainset[0][0]은 이미지이고 trainset[0][1]은 라벨이다. 

print(trainset[0][0].size())

trainset 

torch.Size([3, 16, 16])


일반적인 컬러 사진은 RGB 이미지이기 떄문에 채널이 3개이고 (너비)x(높이)x(채널 수) 로 크기가 표현된다. 

하지만 파이토치에서는 이미지 한 장이 (채널 수) x (너비) x (높이) 로 표현되니 주의! 

In [ ]:
# DataLoader는 데이터를 미니 배치 형태로 만들어 준다.
# 따라서 배치 사이즈 및 셔플 여부 등을 선택해야 한다.

trainloader = DataLoader(trainset, batch_size=50, shuffle= True)
testloader = DataLoader(testset, batch_size=50, shuffle= False)

In [ ]:
len (trainloader)

# CIFAR10의 학습 이미지는 50,000장이고 배치사이즈가 50장이므로 배치갯수는 1000개 이다 

1000

In [ ]:
# iter, next를 이용해 일부 데이터를 확인할 수 있다.
dataiter = iter(trainloader)
images, labels = dataiter.next()

#일반적으로 학습 데이터는 4차원 형태로 모델에서 사용된다.
# (배치크기) x (채널 수) x (너비) x (높이)

print(images.size())

torch.Size([50, 3, 16, 16])


# 3.2 같은 클래스 별로 폴더를 정리한 경우

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
pwd

'/content'

In [ ]:
!cd /content/gdrive/MyDrive/Pytorch

In [ ]:
# 데이터를 같은 클래스끼리 폴더를 미리 나눠줘야 됨 
# ex) class 폴더안에 tiger, lion 폴더를 각각 만든다 

transf = tr.Compose([tr.Resize(128), tr.ToTensor()])
trainset=torchvision.datasets.ImageFolder(root='/content/gdrive/MyDrive/Pytorch/class', transform=transf)
trainloader = DataLoader(trainset,batch_size=1, shuffle= True)

# 3.3 정형화 되지 않은 커스텀 데이터 불러오기(3.2 사용할 수 없을 때)

In [ ]:
#32x32 컬러 이미지와 라벨이 각각 100장이 있다고 가정한다.

train_images = np.random.randint(256,size=(100,32,32,3)) # 이미지 수 x 너비 x 높이 x 채널 수
train_labels = np.random.randint(2,size=(100,1)) #라벨 수 

print(train_images.shape, train_labels.shape)

(100, 32, 32, 3) (100, 1)


In [ ]:
# 양식!!! 
'''
from torch.utils.data import Dataset

class Mydataset(Dataset):

  def __init__(Self):
  def __getitem__(self, index):
  def __len__(self):
'''
from torch.utils import Dataset

class TensorData(Dataset):

  def __init__(self,x_data, y_data):
    self.x_data = torch.FloatTensor(x_data) # 이미지 데이터를 FloatTensor로 변형 
    self.x_data = self.x_data.permute(0,3,1,2) # 이미지 수 x 너비 x 높이 x 채널수 >>>>> 배치크기 x 채널 수 x 너비 x 높이   ## 이렇게 차원 위치 바꿔준다
    self.y_data = torch.LongTensor(y_data) #라벨 데이터를 LongTensor로 변형
    self.len = self.y_data.shape[0] # 클래스 내의 들어 온 데이터 개수 

  def __getitem__(self, index):
    return self.x_data[index], self.y_data[index] # 뽑아낼 데이터 적어준다 

  def __len__(self):
    return self.len # 클래스 내에 들어 온 데이터 개수

In [ ]:
train_data = TensorData(train_images,train_labels)
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)

# 3.4 커스텀 데이터 + 커스텀 전처리 

In [ ]:
import torch 
import torchvision.transforms as tr 
from torch.utils.data import DataLoader, Dataset 
import numpy as np 

In [ ]:
# 32x32 이미지 100장 
train_images = np.random.randint(256, size= (100,32,32,3))
train_labels = np.random.randint(2,size=(100,1))

In [32]:
class TensorData(Dataset):

  def __init__(self,x_data, y_data, transform= None):
    self.x_data = x_data # 이미지 데이터를 FloatTensor로 변형 
    self.y_data = y_data # 이미지 수 x 너비 x 높이 x 채널수 >>>>> 배치크기 x 채널 수 x 너비 x 높이   ## 이렇게 차원 위치 바꿔준다
    self.transform = transform
    self.len = len(y_data) # 클래스 내의 들어 온 데이터 개수 

  def __getitem__(self, index):
    sample = self.x_data[index], self.y_data[index]

    if self.transform: 
      sample= self.transform(sample) #self.transform이 none이 아니라면 전처리 작업실행! 


    return sample # 뽑아낼 데이터 적어준다 

  def __len__(self):
    return self.len # 클래스 내에 들어 온 데이터 개수

In [40]:
# 전처리 기술을 직접 만들어보자 
# 위 기본 양식과 같이 사용하기 위해 call 함수를 사용한다 
# def __call__ 내의 원하는 전처리 작업을 프로그래밍 할 수 있다. 

# 1. 텐서변환 
class ToTensor:
  def __call__(self,sample):
    inputs, labels = sample
    inputs = torch.FloatTensor(inputs)
    inputs = inputs.permute(2,0,1)
    return inputs, torch.LongTensor(labels)

# 2. 선형식
class LinearTensor: 


  def __init__(self,slope=1, bias=0):
    self.slope = slope
    self.bias = bias

  def __call__(self,sample):
    inputs,labels = sample
    inputs = self.slope*inputs + self.bias
    return inputs, labels


In [41]:
trans = tr.Compose([ToTensor(), LinearTensor(2,5)]) #텐서변환 후 선형식 2x+5 연산
dataset1 = TensorData(train_images, train_labels,transform=trans)
train_loader1 = DataLoader(dataset1,batch_size= 10, shuffle=True)


In [42]:
dataiter1 = iter(train_loader1)
images1, labels1 = dataiter1.next()
print(images1.size())

torch.Size([10, 3, 32, 32])


# 3.5 커스텀 데이터 + torchvision.transforms 전처리 

In [43]:
class MyTransform:
  
  def __call__(self, sample):
    inputs, labels = sample
    inputs = torch.FloatTensor(inputs)
    inputs = torch.permute(2,0,1) # call 함수는 이미지를 1장씩 불러와서 처리하므로 인덱스가 0,1,2 밖에 없다 
    labels = torch.FloatTensor(labels)

    transf = tr.Compose([tr.ToPILImage(), tr.Resize(128), tr.ToTensor(), tr.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
    final_output = transf(input)

    return final_output, labels

In [ ]:
dataset2 = TensorData(train_images, train_labels, transform= MyTransform())
train_loader2 = DataLoader(dataset1)